# Python内置函数

- Python有大量的内置函数(built-in function)，比如`abs`
- Python把变量类型转换视为一种函数，有`int()/float()/str()/bool()/hex()`等

# Python函数的定义

- Python中函数定义以def关键字开头
- 函数不需要显示指定返回值的类型，同样不需要指定输入参数的类型
- 函数可以返回多个值，本质上是返回了一个tuple
- 函数体中如果只有`pass`，则表示为一个空的函数，`pass`也可以用于其他语句块中. 可以把`pass`看为一个占位符，代码后续再补充。
- 当函数显式写`return`，或者直接使用`return`没有接变量时，返回的则是`None`类型。

In [1]:
# 函数需要能够检查参数的合法性
# 函数参数的个数检查是编译器在编译期检查的
def my_abs(x):
    if not isinstance(x, (int, float)):
        raise TypeError('bad operand type')
    if x > 0:
        return x
    else:
        return -x
print(my_abs(42))
print(my_abs(-3.14))
print(my_abs('ABC'))

42
3.14


TypeError: bad operand type

# 默认参数

- 默认参数在定义的时候，必须放到非默认参数后面
- 默认参数的定义原则，一般是变化小，大部分调用中不需要改变。
- 使用默认参数可以在扩展函数接口的同时，就保证了向前兼容。
- 默认参数使用起来是有坑的，主要是对默认参数运行机制的理解。

这个点的核心是：**一个函数参数的默认值，仅仅在该函数定义的时候，被赋值一次**，　所以默认参数一旦被赋值，后续无论调用多少次，都不会再重新赋值了。

In [2]:
# 编译的时候add_end的参数就指向了一个分配好的变量
def add_end(L=[]):
    L.append('END')
    return L

print(add_end())
print(add_end()) # 这一次调用，结果并不符合预期，因为默认参数：L并没有重新初始化为[]

['END']
['END', 'END']


所以在使用默认参数时，要注意默认参数的初始值一定要使用不可变的对象，如字面值、str、tuple、None等。上面`add_end`的一种正确的写法，如下：

In [3]:
def add_end(L=None):
    if L is None:
        L = []
    L.append('END')
    return L
print(add_end())
print(add_end())

['END']
['END']


# 可变参数

- 当前我们不确定函数参数的个数时，可以使用可变参数
- 可变参数在定义上，在参数名前加上`*`
- 当我们想把一个List或Tuple的每个元素传入一个可变参数的函数时，可以直接在变量前加`*`来进行元素展开

In [4]:
def my_print(*args):
    print(type(args))
    for v in args:
        print(v)

my_print([1,2,3], "some str", 42) 
my_print(*["Hello", "world", "!"])

<class 'tuple'>
[1, 2, 3]
some str
42
<class 'tuple'>
Hello
world
!


从上面代码的打印中可以看出来，函数内部实际是把可变参数当成一个`tuple`来处理的。

# 关键字参数

可变参数允许你传入0个或任意个参数，这些可变参数在函数调用时自动组装为一个tuple。而关键字参数允许你传入0个或任意个含参数名的参数，这些关键字参数在函数内部自动组装为一个dict。

In [5]:
def person(name, age, **kw):
    print('name:', name, 'age:', age, 'other:', kw)

# 一般的调用方式
person('yansheng', 30, email='ronnyyoung@qq.com', address='ShenZhen')
# 使用字典展开的调用方式
extra_info = {'email': 'ronnyyoung@qq.com', 'address': 'ShenZhen'}
person('yansheng', 30, **extra_info)

name: yansheng age: 30 other: {'email': 'ronnyyoung@qq.com', 'address': 'ShenZhen'}
name: yansheng age: 30 other: {'email': 'ronnyyoung@qq.com', 'address': 'ShenZhen'}


所以，对于任意函数，都可以通过类似`func(*args, **kw)`的形式调用它，无论它的参数是如何定义的。

我们来想一下关键字参数的使用场景。函数的调用者可以传入任意不受限制的关键字参数（可能针对某一类功能，把可能想到的参数都传递进去）。至于到底传入了哪些，就需要在函数内部通过kw检查。

In [6]:
# 函数内部只解析email和address, phone被自动忽略了
def person(name, age, **kw):
    print('name:', name, 'age:', age)
    if 'email' in kw:
        print('email:', kw['email'])
    if 'address' in kw:
        print('address:', kw['address'])
person('yansheng', 30, email='ronnyyoung@qq.com', address='ShenZhen', phone='12345678')

name: yansheng age: 30
email: ronnyyoung@qq.com
address: ShenZhen


# 递归函数

函数内部可以调用函数，如果函数内部调用的是自身，则称为递归调用，下面示例说明用递归函数求阶乘。

In [7]:
def fact(n):
    if n == 1:
        return 1
    return n * fact(n - 1)
fact(5)

120

递归函数虽然代码写起来非常的简明易懂，但是在计算机中使用堆栈来实现函数调用，所以函数调用如果嵌套太深，会发生栈溢出。

In [8]:
fact(100000)

RecursionError: maximum recursion depth exceeded in comparison

避免暴栈的方法是使用尾递归。尾递归是指，在函数返回的时候，调用自身本身，并且，return语句不能包含表达式。这样，编译器或者解释器就可以把尾递归做优化，使递归本身无论调用多少次，都只占用一个栈帧，不会出现栈溢出的情况。

# 函数式编程

函数是Python内建支持的一种封装，我们通过把大段代码拆成函数，通过一层一层的函数调用，就可以把复杂任务分解成简单的任务，这种分解可以称之为**面向过程的程序设计**。函数就是面向过程的程序设计的基本单元。

而函数式编程（请注意多了一个“式”字）——Functional Programming，虽然也可以归结到面向过程的程序设计，但其思想更接近数学计算。

我们首先要搞明白计算机（Computer）和计算（Compute）的概念。

在计算机的层次上，CPU执行的是加减乘除的指令代码，以及各种条件判断和跳转指令，所以，汇编语言是最贴近计算机的语言。

而计算则指数学意义上的计算，越是抽象的计算，离计算机硬件越远。

对应到编程语言，就是越低级的语言，越贴近计算机，抽象程度低，执行效率高，比如C语言；越高级的语言，越贴近计算，抽象程度高，执行效率低，比如Lisp语言。

函数式编程就是一种抽象程度很高的编程范式，纯粹的函数式编程语言编写的函数没有变量，因此，任意一个函数，只要输入是确定的，输出就是确定的，这种纯函数我们称之为没有**副作用(无状态)**。而允许使用变量的程序设计语言，由于函数内部的变量状态不确定，同样的输入，可能得到不同的输出，因此，这种函数是有副作用的。

**函数式编程的一个特点就是，允许把函数本身作为参数传入另一个函数，还允许返回一个函数！**

Python对函数式编程提供部分支持。由于Python允许使用变量，因此，Python不是纯函数式编程语言。

## 函数本身是一种对象

Python中的函数本身就是一种对象，可以赋值给变量，拿内置函数`abs()`举例，`abs(10)`是一个函数调用，而`abs`本身是一个函数对象，可以给普通的变量赋值。

In [9]:
f = abs
f(-10) # f是一个函数对象，可以进行函数调用

10

既然函数本身是一个对象，可以像其他变量一样进行赋值，那也可以作为函数的参数

In [10]:
def add(a, b, f):
    return f(a) + f(b)
add(-10, 3, abs)

13

像上面这样参数有函数对象的函数，我们称之为**高阶函数**

## 高阶函数举例

### map/reduce

map的参数接收一个函数和一个可迭代对象(Iterable)，返回一个迭代器(Iterator)

In [11]:
def square(x):
    return x * x
L = [1,2,3,4]
r = map(square, L)
print(list(r))

[1, 4, 9, 16]


`reduce`把一个函数作用在一个序列`[x1, x2, x3, ...]`上，这个函数必须接收两个参数，`reduce`把结果继续和序列的下一个元素做累积计算。效果就像：
```bash
reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)
```

In [12]:
from functools import reduce
def add(x, y):
    return x + y
r = reduce(add, L)
print(r)

10


### filter

和`map()`类似，`filter()`也接收一个函数和一个序列。和`map()`不同的是，`filter()`把传入的函数依次用于每个元素，然后根据返回值是`True`还是`False`决定保留还是丢弃元素。

下面写一个过滤字符串中包含了"er"的字符串

In [13]:
def include_er(s):
    return  s.find('er') != -1
list(filter(include_er, ['more', 'super', 'taller', 'abc']))

['super', 'taller']

可见用`filter()`这个高阶函数，关键在于正确实现一个“筛选”函数。

注意到`filter()`函数返回的是一个`Iterator`，也就是一个惰性序列，所以要强迫`filter()`完成计算结果，需要用`list()`函数获得所有结果并返回`list`。

### 用filter求素数

计算素数的一个方法是埃氏筛法，它的算法理解起来非常简单：

首先，列出从2开始的所有自然数，构造一个序列：

2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...

取序列的第一个数2，它一定是素数，然后用2把序列的2的倍数筛掉：

3, ~~4~~, 5, ~~6~~, 7, ~~8~~, 9, ~~10~~, 11, ~~12~~, 13, ~~14~~, 15, ~~16~~, 17, ~~18~~, 19, ~~20~~, ...

取新序列的第一个数3，它一定是素数，然后用3把序列的3的倍数筛掉：

5, ~~6~~, 7, ~~8~~, ~~9~~, ~~10~~, 11, ~~12~~, 13, ~~14~~, ~~15~~, ~~16~~, 17, ~~18~~, 19, ~~20~~, ...


取新序列的第一个数5，然后用5把序列的5的倍数筛掉：

7, ~~8~~, ~~9~~, ~~10~~, 11, ~~12~~, 13, ~~14~~, ~~15~~, ~~16~~, 17, ~~18~~, 19, ~~20~~, ...

不断筛下去，就可以得到所有的素数。

用Python来实现这个算法，可以先构造一个从3开始的奇数序列：

In [14]:
# 定义一个奇数序列的生成器
def _odd_iter():
    n = 1
    while True:
        n = n + 2
        yield n

# 定义一个筛选函数
def _not_divisible(n):
    return lambda x: x % n > 0

# 下面的生成过程类型于
# filter(...filter(_not_divisible(7),filter(_not_divisible(5),filter(_not_divisible(3), it)))...)
def primes():
    yield 2
    it = _odd_iter()
    while True:
        n = next(it)
        yield n
        it = filter(_not_divisible(n), it)
     
    
for n in primes():
    if n < 100:
        print(n, end=',')
    else:
        break

2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,

### sorted

排序也是在程序中经常用到的算法。无论使用冒泡排序还是快速排序，排序的核心是比较两个元素的大小。如果是数字，我们可以直接比较，但如果是字符串或者两个dict呢？直接比较数学上的大小是没有意义的，因此，比较的过程必须通过函数抽象出来。

Python内置的`sorted()`函数就可以对list进行排序，此外，`sorted()`函数也是一个高阶函数，它还可以接入一个`key`函数来实现自定义的排序。比如，我们想对一些字符串进行排序，但是想忽略他们的大小写。

In [15]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower)

['about', 'bob', 'Credit', 'Zoo']

要进行反向排序，不必改动key函数，可以传入第三个参数`reverse=True`

In [16]:
sorted(['bob', 'about', 'Zoo', 'Credit'], key=str.lower, reverse=True)

['Zoo', 'Credit', 'bob', 'about']

**`sorted()`也是一个高阶函数。用sorted()排序的关键在于实现一个映射函数。**

# 装饰器

装饰器可以让我们在不需要修改某个函数的源码的情况下，来丰富函数的功能，比如加一些入口与出口日志，记时操作等。

本质上decorator就是一个返回函数的高阶函数。下面我们为一个函数定义一个入口打印日志的装饰器。

In [8]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s()' % (func.__name__))
        return func(*args, **kw)
    return wrapper
@log
def my_add(x, y):
    return x + y
print(my_add(1, 2))

call my_add()
3


上面的实现实际上相当于执行了下面的代码，由于log是一个decorator，它返回一个函数，所以原来的my_add()函数仍然存在，而log_my_add是一个新的函数。

In [9]:
def my_add(x, y):
    return x + y
log_my_add = log(my_add)
print(log_my_add(1, 2))

call my_add()
3


当装饰器本身需要传入参数的时候，地就需要编写一个返回decorator的高阶函数，写出来会更复杂。

In [10]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s  %s()' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

@log('execute')
def my_add(x, y):
    return x + y
print(my_add(1, 2))

execute  my_add()
3


上的代码的执行效果，相当于是执行了`my_add=log('execute')(my_add)`

上面的实现有个缺点就是，所有被包装过后的函数，它的`__name__`值都返回的是`wrapper`，我们可以通过下面的方法来解决。

In [11]:
print(my_add.__name__)

wrapper


In [12]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s()' % (func.__name__))
        return func(*args, **kw)
    return wrapper
@log
def my_add(x, y):
    return x + y
print(my_add.__name__)

my_add


下面定义了一个装饰器，它可以为任意函数调用结束时，打印函数的执行时间。

In [16]:
import time

In [23]:
def metric(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        start = time.time()
        value = func(*args, **kw)
        stop = time.time()
        print('execute %s(), cost %.3f ms' % (func.__name__, 1000 * (stop - start)))
        return value
    return wrapper
     
@metric
def sleep_second(s):
    time.sleep(s)

sleep_second(3)

execute sleep_second(), cost 3001.805 ms


In [20]:
help(time.time)

Help on built-in function time in module time:

time(...)
    time() -> floating point number
    
    Return the current time in seconds since the Epoch.
    Fractions of a second may be present if the system clock provides them.



# 偏函数

偏函数类似于C++中的`bind`函数，支持将一些复杂的函数中的某些参数固定住，使得其变为一个新的简单的函数。

比如类型转换函数`int()`支持，通过参数`base`来选择对字符串进行转换时，选择的进制

In [17]:
int('1001')

1001

In [18]:
int('1001', base=2)

9

如果我们想要一个调用起来简单的对二进制字符串进行转换的函数，我们可以这样做：

In [19]:
def int2(s):
    return int(s, base=2)
int2('1001')

9

`functools.partial`就是帮助我们创建一个偏函数的，不需要我们自己定义`int2()`，可以直接使用下面的代码创建一个新的函数`int2`：

In [20]:
import functools
int2 = functools.partial(int, base=2)
int2('1001')

9

所以，简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

`int2('1001')`类似于调用
```python
kw = {'base':2}
int('1001', **kw)
```